# Optuna: A hyperparameter optimization framework

* *In This Kernel I will use the amazing **Optuna** to find the best hyparameters of our models*

**So, Optuna is an automatic hyperparameter ptimization software framework, particularly designed for machine learning. It features an imperative, define-by-run style user API. The code written with Optuna enjoys high modularity, and the user of Optuna can dynamically construct the search spaces for the hyperparameters.** 
* To learn more about Optuna check this [link](https://optuna.org/)

# Basic Concepts
So, We use the terms study and trial as follows:
* Study: optimization based on an objective function
* Trial: a single execution of the objective function

In [2]:
#!pip install optuna 
import optuna

In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')
#sub = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

In [4]:
train.head()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
0,1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
1,3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
2,4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
3,5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
4,8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [5]:
columns = [col for col in train.columns.to_list() if col not in ['id','target']]

In [6]:
data=train[columns]
target=train['target']

## Let's build our optimization function using optuna

### This function uses XgboostRegressor model, takes 
* the data
* the target
* trial(How many executions we will do) and return the  
### and returns
* RMSE(Root Mean Squared Rrror)

## Notes:
* Note that I used some XGBoostRegressor hyperparameters from Xgboost official site. 
* So if you like to add more parameters or change them, check this [link](https://xgboost.readthedocs.io/en/latest/parameter.html) 
* Also I used early_stopping_rounds to avoid overfiting
* to speedup the training process enable the gpu or comment the first param argument(the training process will takes a lot of time by just using the cpu 😩) 

In [7]:
def objective(trial,data=data,target=target):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
    param = {
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 4000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

## All thing is ready So let's start 🏄‍
* Note that the objective of our fuction is to minimize the RMSE that's why I set direction='minimize'
* you can vary n_trials(number of executions) 

In [9]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-01-17 22:26:06,273] A new study created in memory with name: no-name-a556440d-afe7-46bd-b005-86e4eae66339
[I 2021-01-17 22:27:18,217] Trial 0 finished with value: 0.6945923734716385 and parameters: {'lambda': 0.0017177103549183793, 'alpha': 0.003988562114626481, 'colsample_bytree': 0.8, 'subsample': 1.0, 'learning_rate': 0.009, 'max_depth': 9, 'random_state': 2020, 'min_child_weight': 253}. Best is trial 0 with value: 0.6945923734716385.
[I 2021-01-17 22:29:12,446] Trial 1 finished with value: 0.6940941378096152 and parameters: {'lambda': 0.002386980060622093, 'alpha': 0.057147467523410884, 'colsample_bytree': 1.0, 'subsample': 0.5, 'learning_rate': 0.008, 'max_depth': 20, 'random_state': 2020, 'min_child_weight': 256}. Best is trial 1 with value: 0.6940941378096152.
[I 2021-01-17 22:31:08,624] Trial 2 finished with value: 0.6944582812854488 and parameters: {'lambda': 0.6206525538202995, 'alpha': 4.279277945699177, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.0

KeyboardInterrupt: 

In [ ]:
study.trials_dataframe()

# Let's do some Quick Visualization for Hyperparameter Optimization Analysis
### Optuna provides various visualization features in optuna.visualization to analyze optimization results visually

In [ ]:
#plot_optimization_histor: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
'''plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
went and which parts of the space were explored more.'''
optuna.visualization.plot_slice(study)

In [ ]:
#plot_contour: plots parameter interactions on an interactive chart. You can choose which hyperparameters you would like to explore.
optuna.visualization.plot_contour(study, params=['alpha',
                            #'max_depth',
                            'lambda',
                            'subsample',
                            'learning_rate',
                            'subsample'])

In [ ]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

# Let's create an XgboostRegressor model with the best hyperparameters

In [ ]:
Best_trial {'lambda': 0.0030282073258141168, 'alpha': 0.01563845128469084, 'colsample_bytree': 0.5,
             'subsample': 0.7,'n_estimators': 4000, 'learning_rate': 0.01,'max_depth': 15,
             'random_state': 2020, 'min_child_weight': 257,'tree_method':'gpu_hist'}


# Best_trial = {'lambda': 9.930238209261285, 'alpha': 6.507845870211356, 'colsample_bytree': 0.5,
# 'subsample': 0.6, 'learning_rate': 0.009, 'max_depth': 13,
# 'random_state': 24, 'min_child_weight': 92}

In [ ]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5,random_state=48,shuffle=True)
rmse=[]  # list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(train[columns],train['target']):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = xgb.XGBRegressor(**Best_trial)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds+=model.predict(test[columns])/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    n+=1

In [ ]:
np.mean(rmse)

# Submission

In [ ]:
sub['target']=preds
sub.to_csv('submission.csv', index=False)

# Soon... hyperparameter tuning using Catboost

# # Please If you find this kernel helpful, upvote it to help others see it 😊 😋